# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [106]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [107]:
# your code here
orders = pd.read_csv('Orders.csv')
orders = orders.drop(['Unnamed: 0'], axis=1)
orders.shape

(397924, 13)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [108]:
sums = orders.groupby(['CustomerID','Country']).agg({'amount_spent':sum})
sums

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


In [109]:
# your code here
orders.head()
sums_customer = orders.groupby(['CustomerID','Country']).agg({'amount_spent':sum})
q95 = float(sums_customer.quantile(q=0.95))
vips = sums_customer[sums_customer['amount_spent'] > q95]
vips

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12357,Switzerland,6207.67
12359,Cyprus,6372.58
12409,Switzerland,11072.67
12415,Australia,124914.53
...,...,...
18109,United Kingdom,8052.97
18139,United Kingdom,8438.34
18172,United Kingdom,7561.68


In [110]:
q75 = float(sums_customer.quantile(q=0.75))
preferred = sums_customer[(sums_customer['amount_spent'] > q75) & (sums_customer['amount_spent'] <= q95)]
preferred

,,amount_spent
CustomerID,Country,
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12352,Norway,2506.04
12356,Portugal,2811.43
...,...,...
18259,United Kingdom,2338.60
18260,United Kingdom,2643.20
18272,United Kingdom,3078.58


In [111]:
def status(x):
    if x > q95:
        return 'VIP'
    elif x > q75:
        return 'Preferred'
    else:
        return 'Usual'

sums_customer['status'] = list(map(status, sums_customer['amount_spent']))
sums_customer

,,amount_spent,status
CustomerID,Country,,
12346,United Kingdom,77183.60,VIP
12347,Iceland,4310.00,Preferred
12348,Finland,1797.24,Preferred
12349,Italy,1757.55,Preferred
12350,Norway,334.40,Usual
...,...,...,...
18280,United Kingdom,180.60,Usual
18281,United Kingdom,80.82,Usual
18282,United Kingdom,178.05,Usual


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [123]:
# your code here
def points_q2(row):
    if row['status'] == 'VIP':
        return 1
    else:
        return 0

sums_customer['VIP'] = sums_customer.apply(points_q2, axis=1)
q2 = sums_customer.groupby(['Country']).agg({'VIP':sum})
q2.sort_values(by=['VIP'], ascending=False).head(5)


,VIP
Country,
United Kingdom,178
Germany,11
France,9
Switzerland,3
Spain,2


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [125]:
# your code here
def points_q3(row):
    if row['status'] in ['VIP', 'Preferred']:
        return 1
    else:
        return 0

sums_customer['VIP_Preferred'] = sums_customer.apply(points_q3, axis=1)
q3 = sums_customer.groupby(['Country']).agg({'VIP_Preferred':sum})
q3.sort_values(by=['VIP_Preferred'], ascending=False).head(5)

,VIP_Preferred
Country,
United Kingdom,934
Germany,39
France,29
Belgium,11
Switzerland,9
